### Training with GPT-4.0-mini
By training with GPT-4.0-mini, we aim to achieve better performance and more natural conversations in our chatbot.

### Setting Up OpenAI API Client and Generating a Chat Completion

In this section, we will set up the OpenAI API client and generate a chat completion using the GPT-4 model. The steps include:

1. **Importing Required Libraries**: We import necessary libraries such as `Markdown` and `display` from `IPython.display`, `OpenAI` from `openai`, and `os` and `getpass` for handling environment variables and secure input.

2. **Entering API Key**: We securely input the OpenAI API key using `getpass.getpass()` to avoid exposing the key in the code.

3. **Initializing OpenAI Client**: We initialize the OpenAI client with the provided API key.

4. **Creating a Chat Completion**: We create a chat completion using the `client.chat.completions.create()` method. The model used is `gpt-4o-mini`, and the messages include a system message setting the context and a user message asking about the future of AI.

5. **Displaying the Response**: Finally, we display the response using `Markdown` to render it in a readable format.

In [ ]:
from IPython.display import Markdown, display
from openai import OpenAI
import os
import getpass

openai_api_key = getpass.getpass("Enter your API key: ")
print("API key entered successfully!")


client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a professor at University of San Diego."},
    {"role": "user", "content": "What is the future of AI?"}
  ]
)
display(Markdown(response.choices[0].message.content))

API key entered successfully!


The future of AI is expected to be shaped by several key trends and developments across various sectors:

1. **Enhanced Integration**: AI will become more integrated into our daily lives, enhancing everything from personal assistants to complex business systems. This integration will lead to more seamless user experiences and improved efficiencies.

2. **Ethical AI**: As AI becomes more powerful, the importance of ethics in AI development will grow. Organizations will focus on creating fair, transparent, and accountable AI systems to avoid biases and ensure that the technology benefits all segments of society.

3. **AI and Sustainability**: AI is poised to play a significant role in addressing climate change and sustainability challenges. It can optimize energy consumption, enhance resource management, and facilitate the development of green technologies.

4. **Advancements in Natural Language Processing (NLP)**: We can expect further breakthroughs in NLP, making AI systems more capable of understanding and generating human-like text, which can enhance communication and information retrieval.

5. **AI in Healthcare**: AI's application in healthcare will expand, from diagnostics to personalized medicine. Predictive analytics can lead to better patient outcomes and more efficient healthcare delivery.

6. **Autonomous Systems**: The development of autonomous systems, including self-driving vehicles and drones, will continue. This raises important regulatory, safety, and ethical considerations that will shape how these technologies are implemented.

7. **Human-AI Collaboration**: Instead of replacing jobs, AI will increasingly serve as a tool for augmenting human capabilities. This partnership will enhance productivity and creativity across various professions.

8. **Democratization of AI**: Tools and platforms for developing AI will become more accessible to a wider audience, allowing individuals and small businesses to leverage AI without needing advanced technical skills.

9. **Focus on General AI**: While current AI systems are designed for specific tasks (narrow AI), research into artificial general intelligence (AGI) will continue, aiming to create AI that can understand, learn, and apply knowledge across a wide range of tasks similar to human intelligence.

10. **Regulatory Frameworks**: As AI technologies evolve, governments and international bodies will likely establish new regulations to govern their development and use. This could help manage risks associated with AI, including privacy concerns and job displacement.

Overall, the future of AI holds immense potential for innovation and improvement in many areas, but it also requires careful consideration of ethical implications and societal impacts. Balancing progress with responsibility will be crucial in shaping a positive future for AI.

### Fine-Tuning an OpenAI Model and the Use of JSONL Files

In this section, we will walk through the process of fine-tuning an OpenAI model using a dataset of movie conversations. We will also discuss the importance of converting data to JSONL format and what JSONL files are.

#### Fine-Tuning an OpenAI Model

Fine-tuning is the process of taking a pre-trained model and training it further on a specific dataset to adapt it to a particular task. This allows the model to learn the nuances and patterns specific to the new dataset, improving its performance on related tasks.

The steps involved in fine-tuning an OpenAI model are as follows:

1. **Prepare the Dataset**: The dataset needs to be in a format that the model can understand. For OpenAI models, this often means converting the data into JSONL (JSON Lines) format.
2. **Upload the Dataset**: The prepared dataset is uploaded to OpenAI's servers.
3. **Create a Fine-Tuning Job**: A fine-tuning job is created using the uploaded dataset. This job specifies the model to be fine-tuned, the dataset to use, and any hyperparameters for the training process.
4. **Monitor the Fine-Tuning Process**: The status of the fine-tuning job can be monitored to ensure it is progressing as expected.
5. **Retrieve the Fine-Tuned Model**: Once the fine-tuning process is complete, the fine-tuned model can be retrieved and used for inference.

#### Why Convert Data to JSONL Files?

JSONL (JSON Lines) is a convenient format for storing structured data where each line is a valid JSON object. This format is particularly useful for machine learning tasks for several reasons:

- **Line-by-Line Processing**: Each line in a JSONL file represents a separate JSON object, making it easy to process large datasets line by line without loading the entire file into memory.
- **Flexibility**: JSONL files can store complex nested structures, making them suitable for a wide range of data types.
- **Compatibility**: Many machine learning frameworks and tools, including OpenAI's API, support JSONL format, making it a standard choice for data interchange.


In [ ]:
import os
import re
import json
import pandas as pd
from sklearn.model_selection import train_test_split

# Define file paths
movie_lines_file = "movie_lines.txt"
movie_conversations_file = "movie_conversations.txt"

# Step 1: Load the lines from movie_lines.txt
lines_dict = {}
with open(movie_lines_file, 'r', encoding='iso-8859-1') as file:
    for line in file:
        parts = line.strip().split(" +++$+++ ")
        if len(parts) == 5:  # Ensuring the correct format
            line_id, character_id, movie_id, character_name, text = parts
            lines_dict[line_id] = text

# Step 2: Load conversations from movie_conversations.txt
conversations = []
with open(movie_conversations_file, 'r', encoding='iso-8859-1') as file:
    for line in file:
        parts = line.strip().split(" +++$+++ ")
        if len(parts) == 4:  # Ensuring the correct format
            line_ids_str = parts[3]
            line_ids = re.findall(r"L[0-9]+", line_ids_str)  # Extracting line IDs
            conversation = [lines_dict[line_id] for line_id in line_ids if line_id in lines_dict]
            if len(conversation) > 1:  # Only add conversations with multiple lines
                conversations.append(conversation)

# Prepare the data for training
jsonl_data = []
for conversation in conversations:
    jsonl_data.append({
        "messages": [
            {"role": "system", "content": "Engage in a conversation based on movie dialogues."},
            {"role": "user", "content": conversation[0]},  # First line as user input
            {"role": "assistant", "content": conversation[1]}  # Second line as assistant response
        ]
    })

# Convert to DataFrame for easier handling
df = pd.DataFrame(jsonl_data)

# Split the data into training and validation sets (80% train, 20% validation)
train_data, validation_data = train_test_split(df, test_size=0.2, random_state=42)

def save_to_jsonl(data, output_file_path):
    # Save to JSONL format
    with open(output_file_path, 'w') as f:
        for _, row in data.iterrows():
            f.write(json.dumps(row.to_dict()) + '\n')

# Save the training and validation sets to separate JSONL files
train_output_file_path = 'movie_conversation_train.jsonl' 
validation_output_file_path = 'movie_conversation_validation.jsonl'

save_to_jsonl(train_data, train_output_file_path)
save_to_jsonl(validation_data, validation_output_file_path)

print(f"Training dataset saved to {train_output_file_path}")
print(f"Validation dataset saved to {validation_output_file_path}")

Training dataset saved to movie_conversation_train.jsonl
Validation dataset saved to movie_conversation_validation.jsonl


### Uploading Training and Validation Files for Fine-Tuning

In this section, we will upload the training and validation datasets to OpenAI's servers to prepare for fine-tuning the model. The datasets are in JSONL format, which is required for the fine-tuning process.

The steps involved are as follows:

1. **Upload Training File**: We use the `client.files.create()` method to upload the training dataset. The file is opened in binary read mode (`"rb"`) and the purpose is set to `"fine-tune"`.

2. **Upload Validation File**: Similarly, we upload the validation dataset using the same method.

3. **Print File Information**: After uploading, we print the information of both the training and validation files to confirm successful uploads.

In [ ]:
train_file = client.files.create(
  file=open(train_output_file_path, "rb"),
  purpose="fine-tune"
)

valid_file = client.files.create(
  file=open(validation_output_file_path, "rb"),
  purpose="fine-tune"
)

print(f"Training file Info: {train_file}")
print(f"Validation file Info: {valid_file}")

Training file Info: FileObject(id='file-Hs5HujD5LxnH63f5ewQWK0o7', bytes=18403816, created_at=1729157290, filename='movie_conversation_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Validation file Info: FileObject(id='file-k5g9mrkMXhZvuBwYnALcDFaX', bytes=4602260, created_at=1729157293, filename='movie_conversation_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


### Creating a Fine-Tuning Job

In this section, we will create a fine-tuning job using the OpenAI API. The fine-tuning job will use the training and validation datasets we previously uploaded. We will specify the model to be fine-tuned and set the hyperparameters for the training process.

The steps involved are as follows:

1. **Specify Training and Validation Files**: We use the IDs of the training and validation files that were uploaded to OpenAI's servers.
2. **Set Model and Hyperparameters**: We specify the model to be fine-tuned (`gpt-4o-mini-2024-07-18`) and set the hyperparameters, including the number of epochs, batch size, and learning rate multiplier.
3. **Create Fine-Tuning Job**: We create the fine-tuning job using the `client.fine_tuning.jobs.create()` method.
4. **Retrieve Job ID and Status**: After creating the job, we retrieve the job ID and status to monitor the fine-tuning process.


In [ ]:
model = client.fine_tuning.jobs.create(
  training_file=train_file.id, 
  validation_file=valid_file.id,
  model="gpt-4o-mini-2024-07-18", 
  hyperparameters={
    "n_epochs": 3,
	"batch_size": 3,
	"learning_rate_multiplier": 0.3
  }
)
job_id = model.id
status = model.status

print(f'Fine-tuning model with jobID: {job_id}.')
print(f"Training Response: {model}")
print(f"Training Status: {status}")

Fine-tuning model with jobID: ftjob-nJMvXlFf73BUJKCrX7jkpuLO.
Training Response: FineTuningJob(id='ftjob-nJMvXlFf73BUJKCrX7jkpuLO', created_at=1729157330, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=0.3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-pZiumcjzaUnCXxgMZCyV8P9C', result_files=[], seed=537079073, status='validating_files', trained_tokens=None, training_file='file-Hs5HujD5LxnH63f5ewQWK0o7', validation_file='file-k5g9mrkMXhZvuBwYnALcDFaX', estimated_finish=None, integrations=[], user_provided_suffix=None)
Training Status: validating_files


In [ ]:
import time
job = client.fine_tuning.jobs.retrieve("ftjob-nJMvXlFf73BUJKCrX7jkpuLO")

# Display the job details in a readable format
print(f"Job ID: {job.id}")
print(f"Model: {job.model}")
print(f"Fine-Tuned Model: {job.fine_tuned_model}")
print(f"Status: {job.status}")
print(f"Created At: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(job.created_at))}")
print(f"Finished At: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(job.finished_at))}")
print(f"Training File: {job.training_file}")
print(f"Validation File: {job.validation_file}")
print(f"Hyperparameters: {job.hyperparameters}")
print(f"Trained Tokens: {job.trained_tokens}")
print(f"Result Files: {job.result_files}")
print(f"Error: {job.error.message if job.error.code else 'None'}")

Job ID: ftjob-nJMvXlFf73BUJKCrX7jkpuLO
Model: gpt-4o-mini-2024-07-18
Fine-Tuned Model: ft:gpt-4o-mini-2024-07-18:personal::AJWkdy4H
Status: succeeded
Created At: 2024-10-17 09:28:50
Finished At: 2024-10-18 02:11:29
Training File: file-Hs5HujD5LxnH63f5ewQWK0o7
Validation File: file-k5g9mrkMXhZvuBwYnALcDFaX
Hyperparameters: Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=0.3)
Trained Tokens: 9891729
Result Files: ['file-4AFc3bMpt4K0KE90P6g46Xvu']
Error: None


In [ ]:
import time

# Function to retrieve and print fine-tuning job details including loss
def check_fine_tuning_job_status(client, job_id):
    while True:
        # Retrieve the state of the fine-tuning job
        job = client.fine_tuning.jobs.retrieve(job_id)

        # Print relevant details
        print(f"Status: {job.status}")
        print(f"Hyperparameters: {job.hyperparameters}")
        print(f"Model: {job.model}")
        print(f"Training File: {job.training_file}")
        print(f"Validation File: {job.validation_file}")

        # Check for error status
        if job.error.code is not None:
            print(f"Error: {job.error.message}")
            break

        # Check if the job has finished
        if job.status in ['succeeded', 'failed', 'cancelled']:
            print("Fine-tuning job has completed.")
            break
        
        # Optionally retrieve training metrics like loss
        if job.status in ['training', 'validating_files']:  # Check while in training
            if hasattr(job, 'loss'):
                print(f"Current Loss: {job.loss}")  # Adjust based on actual API response structure

        # Wait for a while before checking again
        print("Waiting for the job to finish...")
        time.sleep(10)  # Check every 10 seconds

job_id = 'ftjob-nJMvXlFf73BUJKCrX7jkpuLO' 
check_fine_tuning_job_status(client, job_id)

print(f"Status: {job.status}")
print(f"Hyperparameters: {job.hyperparameters}")
print(f"Model: {job.model}")

Status: succeeded
Hyperparameters: Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=0.3)
Model: gpt-4o-mini-2024-07-18
Training File: file-Hs5HujD5LxnH63f5ewQWK0o7
Validation File: file-k5g9mrkMXhZvuBwYnALcDFaX
Fine-tuning job has completed.
Status: succeeded
Hyperparameters: Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=0.3)
Model: gpt-4o-mini-2024-07-18


In [ ]:
import time
import requests

# Function to retrieve and print checkpoints for the fine-tuning job
def check_fine_tuning_job_checkpoints(job_id):
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }

    while True:
        # Call the OpenAI API to retrieve checkpoints
        response = requests.get(f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}/checkpoints", headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            print(data)
            # Process and print each checkpoint
            for checkpoint in data['data']:
                step_number = checkpoint['step_number']
                train_loss = checkpoint['metrics'].get('train_loss')
                full_valid_mean_token_accuracy = checkpoint['metrics'].get('train_mean_token_accuracy')
                fine_tuned_model_checkpoint = checkpoint.get('fine_tuned_model_checkpoint')
                # Print relevant details for each checkpoint
                print(f"Step Number: {step_number}")
                print(f"Train Loss: {train_loss}")
                print(f"Mean Token Accuracy: {full_valid_mean_token_accuracy}")
                
                print(f"CheckPoint Name: {fine_tuned_model_checkpoint}")
                print("------")
                
            # Check if there are more checkpoints to retrieve
            if not data.get("has_more", False):
                print("No more checkpoints to retrieve.")
                break
        else:
            print(f"Error retrieving checkpoints: {response.status_code}, {response.text}")
            break

        # Wait before checking again
        print("Waiting for the next checkpoint...")
        time.sleep(10)  # Adjust the wait time as needed

# Call the function to start checking for checkpoints
check_fine_tuning_job_checkpoints(job_id)

{'object': 'list', 'data': [{'object': 'fine_tuning.job.checkpoint', 'id': 'ftckpt_lZX1fMnF4x0Wfo0Ok2d53GFK', 'created_at': 1729217486, 'fine_tuned_model_checkpoint': 'ft:gpt-4o-mini-2024-07-18:personal::AJWkdy4H', 'fine_tuning_job_id': 'ftjob-nJMvXlFf73BUJKCrX7jkpuLO', 'metrics': {'step': 66427, 'train_loss': 1.9935097694396973, 'train_mean_token_accuracy': 0.5277777910232544}, 'step_number': 66427}, {'object': 'fine_tuning.job.checkpoint', 'id': 'ftckpt_nmrDhwfYLautzFgIXyrfhXDK', 'created_at': 1729217398, 'fine_tuned_model_checkpoint': 'ft:gpt-4o-mini-2024-07-18:personal::AJWkdxWX:ckpt-step-66426', 'fine_tuning_job_id': 'ftjob-nJMvXlFf73BUJKCrX7jkpuLO', 'metrics': {'step': 66426, 'train_loss': 2.293053388595581, 'train_mean_token_accuracy': 0.49152541160583496}, 'step_number': 66426}, {'object': 'fine_tuning.job.checkpoint', 'id': 'ftckpt_VzR8v7a5e5YiNXqiCQXQjZ7J', 'created_at': 1729198028, 'fine_tuned_model_checkpoint': 'ft:gpt-4o-mini-2024-07-18:personal::AJWkdVEz:ckpt-step-44284',

In [17]:
import openai

# Retrieve the list of fine-tuning jobs using the client and get the fine-tuned model
result = client.fine_tuning.jobs.list()
fine_tuned_model = "ft:gpt-4o-mini-2024-07-18:personal::AJWkdy4H"  # Use the correct key to get fine-tuned model name
print(f"Using Fine-Tuned Model: {fine_tuned_model}")

# Initialize the conversation history
messages = [
    {"role": "system", "content": "Given a social media post, classify whether it indicates 'stress' or 'non-stress'."}
]

# Chat loop with the fine-tuned model
while True:
    # User input prompt
    user_input = input("You: ")

    # Check for exit condition
    if user_input.lower() in ["exit", "quit", "q"]:
        print("Ending the chat. Goodbye!")
        break

    # Append user message to conversation history
    messages.append({"role": "user", "content": user_input})

    # Call the OpenAI chat completion API using the client object
    response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=messages
    )
    # Retrieve and display the model's response
    assistant_message = response.choices[0].message.content
    print(f"You: {user_input}")
    print(f"Assistant: {assistant_message}")

    # Append the assistant's response to conversation history
    messages.append({"role": "assistant", "content": assistant_message})

Using Fine-Tuned Model: ft:gpt-4o-mini-2024-07-18:personal::AJWkdy4H
You: Hey! You went to see her yesterday?
Assistant: I sure did. I had to explain things to her. I said, we're friends. You know, you just gotta be sure that everybody you care about can get along forever. You can't have bad feelings about people.
You: What she told you?
Assistant: Well, how nervous she was. She wanted to tell me how nervous she was.
You: And why she was nervous?
Assistant: Knowing that she was, you know, on the eve of something. She sure is uptight.
You: Let her rest for sometime
Assistant: I went to pick her up at the train station, she's being so... Nervous.
Ending the chat. Goodbye!
